In [3]:
import requests
import os
# https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
response = requests.get("https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt")
target_file = "files/chapter_three.txt"
os.makedirs(os.path.dirname(target_file), exist_ok=True)
with open(target_file, "wb") as f:
    f.write(response.content)